In [ ]:
import pandas as pd

In [3]:
# Assuming 'train' is your DataFrame
df = pd.read_csv('data.csv').drop(columns=['Ticket Price Avg. USD', 'Avg. Tickets Sold', 'Currency'])
# Convert 'Event Date' to datetime if it's not already
df['Event Date'] = pd.to_datetime(df['Event Date'], errors='coerce')

# Sort the DataFrame by 'Event Date' (ascending order)
df = df.sort_values('Event Date')

# Determine the split index (80% for training, 20% for testing)
split_index = int(0.8 * len(df))

# Split the dataset
train = df.iloc[:split_index]

In [4]:
def fill_missing_values(df, column_name):
    # Clean the column by removing unwanted characters
    df[column_name] = df[column_name].replace({'%': '', ',': ''}, regex=True)
    
    # Convert the column to numeric, coercing errors to NaN
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    
    # Fill missing values with the mean of the respective 'Venue', ignoring NaN values
    df[column_name] = df.groupby('Venue')[column_name].transform(lambda x: x.fillna(x.mean()))
    
    # If still missing, fill with the global mean
    global_mean = df[column_name].mean()
    df[column_name] = df[column_name].fillna(global_mean)
    
    return df

def preprocess_data(df):
    # Step 1: Convert 'Event Date' to numerical features
    df['Event Date'] = pd.to_datetime(df['Event Date'])
    df['year'] = df['Event Date'].dt.year
    df['month'] = df['Event Date'].dt.month
    df['day'] = df['Event Date'].dt.day
    df['weekday'] = df['Event Date'].dt.weekday
    df['hour'] = df['Event Date'].dt.hour  # Assuming time is included; otherwise, set to 0

    # Drop the 'Event Date' column if no longer needed
    df.drop(columns=['Event Date'], inplace=True)

    # Step 2: Convert currency columns to float
    currency_columns = ['Avg. Gross USD', 'Ticket Price Min USD', 'Ticket Price Max USD']
    for col in currency_columns:
        df[col] = df[col].replace({'\$': '', ',': ''}, regex=True).astype(float)

    # Step 3: Handle missing values in 'Avg. Capacity Sold' using the helper function
    df = fill_missing_values(df, 'Avg. Capacity Sold')
    
    # Step 4: Handle missing values for specific categorical columns
    categorical_columns = ['Support', 'Market', 'Genre', 'Promoter', 'Company Type', 'Venue', 'State']
    for col in categorical_columns:
        df[col] = df[col].fillna('None')
    
    # Step 5: Handle missing values in 'Avg. Event Capacity' using the helper function
    df = fill_missing_values(df, 'Avg. Event Capacity')
    
    return df


In [5]:
preprocess_data(train)

C:\Users\sarah\AppData\Local\Temp\ipykernel_20056\129640061.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Event Date'] = pd.to_datetime(df['Event Date'])
C:\Users\sarah\AppData\Local\Temp\ipykernel_20056\129640061.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['Event Date'].dt.year
C:\Users\sarah\AppData\Local\Temp\ipykernel_20056\129640061.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

,Number of Shows,Headliner,Support,Venue,City,State,Country,Market,Company Type,Promoter,...,Avg. Gross USD,Avg. Event Capacity,Avg. Capacity Sold,Ticket Price Min USD,Ticket Price Max USD,year,month,day,weekday,hour
704977,1,Edwin McCain,Far Too Jones,The Coca-Cola Roxy Theatre,Atlanta,Georgia,United States,Atlanta,Auditorium / Theatre,Alex Cooley / Peter Conlon,...,19854.0,1650,66.0,18.0,18.0,1999,1,1,4,0
704965,2,Asylum Street Spankers,None,Cafe Du Nord,San Francisco,California,United States,San Francisco-Oakland-San Jose,Club,Continental Drift Productions,...,1897.0,271,100.0,7.0,7.0,1999,1,1,4,0
704966,1,Heather Myles,None,Johnny D's Restaurant & Music Club,Somerville,Massachusetts,United States,Boston (Manchester),Club,(In-House Promotion),...,2645.0,300,88.0,10.0,10.0,1999,1,1,4,0
704967,2,Eddie Money,Eric Martin,Konocti Harbor Resort & Spa - Showroom,Kelseyville,California,United States,None,Club,(In-House Promotion) / Greg Bennett,...,19734.5,1000,68.0,29.0,29.0,1999,1,1,4,0
704968,1,Kirk Franklin & Nu Nation,CeCe Winans,The Liacouras Center,Philadelphia,Pennsylvania,United States,Philadelphia,Arena,Rainbow Promotions,...,264250.0,9146,100.0,29.5,35.5,1999,1,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141043,1,The Sweet Remains,None,Sun Valley Opera House,Sun Valley,Idaho,United States,Boise,Auditorium / Theatre,(In-House Promotion),...,13979.0,314,99.0,17.5,70.0,2019,2,6,2,0
141045,1,Sarah Brightman,None,Radio City Music Hall,New York,New York,United States,New York,Auditorium / Theatre,Live Nation,...,399109.0,5943,70.0,50.0,250.0,2019,2,6,2,0
141046,1,2Cellos,None,Moda Center,Portland,Oregon,United States,"Portland, OR",Arena,(In-House Promotion),...,310246.0,5996,68.0,39.5,99.5,2019,2,6,2,0
141012,1,Tracy Byrd,None,Mary Stuart Rogers Theatre,Modesto,California,United States,Sacramento-Stockton-Modesto,Auditorium / Theatre,(In-House Promotion),...,45010.0,1248,84.0,29.0,59.0,2019,2,6,2,0


In [8]:
train.columns

Index(['Number of Shows', 'Headliner', 'Support', 'Venue', 'City', 'State',
       'Country', 'Market', 'Company Type', 'Promoter', 'Genre',
       'Avg. Gross USD', 'Avg. Event Capacity', 'Avg. Capacity Sold',
       'Ticket Price Min USD', 'Ticket Price Max USD', 'year', 'month', 'day',
       'weekday', 'hour'],
      dtype='object')

In [10]:
# For categorical columns, get the top 20 values (top 50 for 'Headliner')
top_headliners = train['Headliner'].value_counts().head(50).index.tolist()
top_support = train['Support'].value_counts().head(20).index.tolist()
top_venue = train['Venue'].value_counts().head(30).index.tolist()
top_city = train['City'].value_counts().head(20).index.tolist()
top_state = train['State'].value_counts().head(20).index.tolist()
top_company_type = train['Company Type'].value_counts().head(20).index.tolist()
top_promoter = train['Promoter'].value_counts().head(20).index.tolist()
top_genre = train['Genre'].value_counts().head(20).index.tolist()

# Filter the DataFrame based on these top values
filtered_train = train[
    train['Headliner'].isin(top_headliners) &
    train['Support'].isin(top_support) &
    train['Venue'].isin(top_venue) &
    train['City'].isin(top_city) &
    train['State'].isin(top_state) &
    train['Company Type'].isin(top_company_type) &
    train['Promoter'].isin(top_promoter) &
    train['Genre'].isin(top_genre)]

In [12]:
# Export the filtered DataFrame to a CSV file
train.to_csv("concert_data.csv", index=False)

In [17]:
train

,Number of Shows,Headliner,Support,Venue,City,State,Country,Market,Company Type,Promoter,...,Avg. Gross USD,Avg. Event Capacity,Avg. Capacity Sold,Ticket Price Min USD,Ticket Price Max USD,year,month,day,weekday,hour
704977,1,Edwin McCain,Far Too Jones,The Coca-Cola Roxy Theatre,Atlanta,Georgia,United States,Atlanta,Auditorium / Theatre,Alex Cooley / Peter Conlon,...,19854.0,1650,66.0,18.0,18.0,1999,1,1,4,0
704965,2,Asylum Street Spankers,None,Cafe Du Nord,San Francisco,California,United States,San Francisco-Oakland-San Jose,Club,Continental Drift Productions,...,1897.0,271,100.0,7.0,7.0,1999,1,1,4,0
704966,1,Heather Myles,None,Johnny D's Restaurant & Music Club,Somerville,Massachusetts,United States,Boston (Manchester),Club,(In-House Promotion),...,2645.0,300,88.0,10.0,10.0,1999,1,1,4,0
704967,2,Eddie Money,Eric Martin,Konocti Harbor Resort & Spa - Showroom,Kelseyville,California,United States,None,Club,(In-House Promotion) / Greg Bennett,...,19734.5,1000,68.0,29.0,29.0,1999,1,1,4,0
704968,1,Kirk Franklin & Nu Nation,CeCe Winans,The Liacouras Center,Philadelphia,Pennsylvania,United States,Philadelphia,Arena,Rainbow Promotions,...,264250.0,9146,100.0,29.5,35.5,1999,1,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141043,1,The Sweet Remains,None,Sun Valley Opera House,Sun Valley,Idaho,United States,Boise,Auditorium / Theatre,(In-House Promotion),...,13979.0,314,99.0,17.5,70.0,2019,2,6,2,0
141045,1,Sarah Brightman,None,Radio City Music Hall,New York,New York,United States,New York,Auditorium / Theatre,Live Nation,...,399109.0,5943,70.0,50.0,250.0,2019,2,6,2,0
141046,1,2Cellos,None,Moda Center,Portland,Oregon,United States,"Portland, OR",Arena,(In-House Promotion),...,310246.0,5996,68.0,39.5,99.5,2019,2,6,2,0
141012,1,Tracy Byrd,None,Mary Stuart Rogers Theatre,Modesto,California,United States,Sacramento-Stockton-Modesto,Auditorium / Theatre,(In-House Promotion),...,45010.0,1248,84.0,29.0,59.0,2019,2,6,2,0
